In [ ]:
import pandas as pd
import os

dir_path = 'data/raw'
symbol = 'usdjpy'
file_type = '1h_raw.csv'
file = os.path.join(dir_path, symbol)

file = file + file_type

df = pd.read_csv(file)

df

,Date,Open,High,Low,Close,Volume
0,2017-08-15 19:00,110.582,110.590,110.505,110.541,4412
1,2017-08-15 20:00,110.541,110.704,110.516,110.666,4102
2,2017-08-15 21:00,110.666,110.684,110.569,110.581,6554
3,2017-08-15 22:00,110.581,110.651,110.578,110.632,3321
4,2017-08-15 23:00,110.632,110.632,110.543,110.592,2256
...,...,...,...,...,...,...
34308,2023-03-31 16:00,132.750,132.878,132.708,132.763,10719
34309,2023-03-31 17:00,132.763,132.962,132.746,132.946,9211
34310,2023-03-31 18:00,132.947,132.948,132.619,132.685,9719
34311,2023-03-31 19:00,132.684,132.781,132.681,132.756,6987


In [11]:
import numpy as np

lookahead_period = 48
threshold_rate = 0.003

df['Future_Close'] = df['Close'].shift(-lookahead_period)
df['Return'] = (df['Future_Close'] / df['Close']) - 1

conditions = [
    (df['Return'] > threshold_rate),
    (df['Return'] < -threshold_rate)
]

choices = [1, -1]
df_len = len(df)
df['Label'] = np.select(conditions, choices, default=0)
labeled_df = df.iloc[:df_len - lookahead_period]

l0 = 0
l1 = 0
l2 = 0
for i in labeled_df['Label']:
    if i == 0:
        l0 += 1
    elif i == 1:
        l1 += 1
    elif i == -1:
        l2 += 1

print(labeled_df)
print(l0, l1, l2)

                   Date     Open     High      Low    Close  Volume  \
0      2017-08-15 19:00  110.582  110.590  110.505  110.541    4412   
1      2017-08-15 20:00  110.541  110.704  110.516  110.666    4102   
2      2017-08-15 21:00  110.666  110.684  110.569  110.581    6554   
3      2017-08-15 22:00  110.581  110.651  110.578  110.632    3321   
4      2017-08-15 23:00  110.632  110.632  110.543  110.592    2256   
...                 ...      ...      ...      ...      ...     ...   
34260  2023-03-29 16:00  132.697  132.698  132.587  132.647   13772   
34261  2023-03-29 17:00  132.646  132.763  132.549  132.578   13555   
34262  2023-03-29 18:00  132.577  132.730  132.553  132.650    9697   
34263  2023-03-29 19:00  132.651  132.855  132.620  132.825    8478   
34264  2023-03-29 20:00  132.824  132.889  132.797  132.842    4719   

       Future_Close    Return  Label  
0           109.488 -0.009526     -1  
1           109.560 -0.009994     -1  
2           109.439 -0.010327 

In [ ]:
filename = f'FTH_{symbol}_w48.csv'
save_path = 'data/processed'

labeled_df.to_csv(os.path.join(save_path, filename), index=False)